In [4]:
from pdfminer.high_level import extract_text
from pdfminer.layout import LAParams
from io import StringIO

def highlight_pdf(pdf_path):
    # Read the PDF file and extract the text
    with open(pdf_path, 'rb') as f:
        text = extract_text(f, laparams=LAParams())
    text = text.split("\n")

    # Find the index of the first two sentences on page 3
    page_start = 0
    page_end = 0
    for i, line in enumerate(text):
        if line.startswith("Page 3"):
            page_start = i
            break
    for i, line in enumerate(text[page_start:], start=page_start):
        if line == "":
            page_end = i
            break

    # Highlight the first two sentences on page 3
    highlighted_text = text[page_start:page_end]
    highlighted_text[0] = ">> " + highlighted_text[0]
    highlighted_text[1] = ">> " + highlighted_text[1]
    text[page_start:page_end] = highlighted_text

    # Write the highlighted text back to a new PDF file
    with open("highlighted.pdf", "wb") as pdf_file:
        pdf_file.write(StringIO("\n".join(text)).getvalue())


ModuleNotFoundError: No module named 'pdfminer'

In [5]:
vorname = "Max"
nachname = "Mustermann"
anrede = "Lieber Herr " vorname + " " + nachname
anrede

SyntaxError: invalid syntax (<ipython-input-5-112cfa56dd3b>, line 3)